In [ ]:
# =============================================================================
# Main Script for CMG Parser _ ver 1.0
# Author: Honggeun Jo
# Date: 08/21/2023 - 
# =============================================================================

# =============================================================================
# Import librarays & Initializing
# =============================================================================
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
import subprocess
import Subfunction as sub
import matplotlib.cm as cm

# Basic info for CMG
CMG =  '"C:\\Program Files\\CMG\\IMEX\\2022.10\\Win_x64\\EXE\\mx202210.exe"'
Current_Working_Dir = "C:/100 Research/cmg_parser"
os.chdir(Current_Working_Dir)
SIM =  "Forward_simulation.dat"
# Total_Number of realizations
No_realization = 10
# Reservoir extent
nx = 28; ny = 28; nz = 20
# number of wells 
No_well_Prod = 4
No_well_Inj = 1
# time steps
Ptime = np.arange(50, 1001, 50)
## Run CMG : subprocess.call([CMG,"-f",SIM])

In [ ]:
# ===a==========================================================================
# Load ensemble
# =============================================================================
Realizations_LogPerm = np.load('ensemble_101.npz')['logPermeability']
Realizations_Porosity = np.load('ensemble_101.npz')['porosity']

print(Realizations_LogPerm.shape, Realizations_Porosity.shape)

In [ ]:
# =============================================================================
# Make directories for ensemble
# =============================================================================
sub.Tstep(Ptime)
for i in range(No_realization) :
    if os.path.exists(f'realization_{str(i)}')==False: 
        os.mkdir(f'realization_{str(i)}')
    # main data file
    shutil.copy('Forward_simulation.dat', f'realization_{str(i)}/Forward_simulation.dat')
    # time steps
    shutil.copy('TSTEP.DAT', f'realization_{str(i)}/TSTEP.DAT')
    # conttol files
    shutil.copy('Control_Inj.DAT', f'realization_{str(i)}/Control_Inj.DAT')
    shutil.copy('Control_Prod.DAT', f'realization_{str(i)}/Control_Prod.DAT')
    # properties
    sub.SetPerm(Realizations_LogPerm[i].T, dir_ = f'realization_{str(i)}')
    sub.SetPoro(Realizations_Porosity[i].T, dir_ = f'realization_{str(i)}')
    

In [ ]:
# =============================================================================
# Generate variables to save results
# =============================================================================
WBHP, WGBP, WOPR, WWPR, WGPR, WCOP, WCWP, WCGP, WWIR, WCWI = sub.null_prod(No_realization, No_well_Prod, No_well_Inj, Ptime)

In [ ]:
# =============================================================================
# Run simulations
# =============================================================================
for i in range(No_realization) :
    os.chdir(f'{Current_Working_Dir}/realization_'+str(i))
    os.system(f'{CMG} -f {SIM}')
    WBHP[i], WGBP[i], WOPR[i], WWPR[i], WGPR[i], WCOP[i], WCWP[i], WCGP[i], WWIR[i], WCWI[i] = sub.read_out(Ptime, 'Forward_simulation.out', No_well_Prod, No_well_Inj)
    print(f'Ensemble simulation for realization_{str(i)} is completed')


In [ ]:
# =============================================================================
# Save and visualize results
# =============================================================================
# WOPR:
plt.figure(figsize =(10,10))
for i_ in range(1,5):
    well_num = i_ -1
    plt.subplot(2,2,i_)
    plt.plot(Ptime,WOPR[i,:,well_num].T, '-', c = [0.2,0.2,0.2], label = 'Inital ensemble',linewidth=0.5);
    for i in range(1,No_realization):    
        plt.plot(Ptime,WOPR[i,:,well_num].T, '-', c = [0.2,0.2,0.2],linewidth=0.5);
    plt.legend()
    plt.title(f'Prod {i_}')
    plt.xlabel('Time, days')
    plt.ylabel('Production rate, STB/day')
plt.suptitle('Well Oil Production Rate')
plt.savefig('WOPR.svg')

    

In [ ]:

# WWPR:
plt.figure(figsize =(10,10))
for i_ in range(1,5):
    well_num = i_ -1
    plt.subplot(2,2,i_)
    plt.plot(Ptime,WWPR[i,:,well_num].T, '-', c = [0.2, 0.2, 0.2], label = 'Inital ensemble',linewidth=0.5);
    for i in range(1,No_realization):    
        plt.plot(Ptime,WWPR[i,:,well_num].T, '-', c = [0.2, 0.2, 0.2],linewidth=0.5);
    plt.legend()
    plt.title(f'Prod {i_}')
    plt.xlabel('Time, days')
    plt.ylabel('Production rate, STB/day')
plt.suptitle('Well water Production Rate')
plt.savefig('WWPR.svg')


In [ ]:

# WCOP:
plt.figure(figsize =(10,10))
for i_ in range(1,5):
    well_num = i_ -1
    plt.subplot(2,2,i_)
    plt.plot(Ptime,WCOP[i,:,well_num].T, '-', c = [0.2, 0.2, 0.2], label = 'Inital ensemble',linewidth=0.5);
    for i in range(1,No_realization):      
        plt.plot(Ptime,WCOP[i,:,well_num].T, '-', c = [0.2, 0.2, 0.2],linewidth=0.5);
    plt.legend()
    plt.title(f'Prod {i_}')
    plt.xlabel('Time, days')
    plt.ylabel('Production amount, MSTB')
plt.suptitle('Well Cumulative Oil Production')
plt.savefig('WCOP.svg')



In [ ]:

# WCWP:
plt.figure(figsize =(10,10))
for i_ in range(1,5):
    well_num = i_ -1
    plt.subplot(2,2,i_)
    plt.plot(Ptime,WCWP[i,:,well_num].T, '-', c = [0.2, 0.2, 0.2], label = 'Inital ensemble',linewidth=0.5);
    for i in range(1,No_realization):      
        plt.plot(Ptime,WCWP[i,:,well_num].T, '-', c = [0.2, 0.2, 0.2],linewidth=0.5);
    plt.legend()
    plt.title(f'Prod {i_}')
    plt.xlabel('Time, days')
    plt.ylabel('Production amount, MSTB')
plt.suptitle('Well Cumulative Water Production')
plt.savefig('WCWP.svg')       